In [1]:
import sqlite3 as sql
import pandas as pd

In [2]:
conn = sql.connect('../USwildfire.sqlite')
fire = pd.read_sql('SELECT * FROM Fires', conn)

In [3]:
type(fire)

pandas.core.frame.DataFrame

In [4]:
fire.shape

(1880465, 39)

In [5]:
fire.head(10)

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape
0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,0511,Plumas National Forest,...,A,40.036944,-121.005833,5.0,USFS,CA,63,063,Plumas,b'\x00\x01\xad\x10\x00\x00\xe8d\xc2\x92_@^\xc0...
1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.933056,-120.404444,5.0,USFS,CA,61,061,Placer,b'\x00\x01\xad\x10\x00\x00T\xb6\xeej\xe2\x19^\...
2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.984167,-120.735556,13.0,STATE OR PRIVATE,CA,17,017,El Dorado,b'\x00\x01\xad\x10\x00\x00\xd0\xa5\xa0W\x13/^\...
3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.559167,-119.913333,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00\x94\xac\xa3\rt\xfa]...
4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.559167,-119.933056,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00@\xe3\xaa.\xb7\xfb]\...
5,6,6,FS-1418849,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.635278,-120.103611,5.0,USFS,CA,5,005,Amador,b'\x00\x01\xad\x10\x00\x00\xf0<~\x90\xa1\x06^\...
6,7,7,FS-1418851,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.688333,-120.153333,5.0,USFS,CA,17,017,El Dorado,b'\x00\x01\xad\x10\x00\x00$o\x996\xd0\t^\xc0h\...
7,8,8,FS-1418854,FED,FS-FIRESTAT,FS,USCASHF,Shasta-Trinity National Forest,0514,Shasta-Trinity National Forest,...,B,40.968056,-122.433889,13.0,STATE OR PRIVATE,CA,None,None,None,b'\x00\x01\xad\x10\x00\x00t)\xe8\xd5\xc4\x9b^\...
8,9,9,FS-1418856,FED,FS-FIRESTAT,FS,USCASHF,Shasta-Trinity National Forest,0514,Shasta-Trinity National Forest,...,B,41.233611,-122.283333,13.0,STATE OR PRIVATE,CA,None,None,None,"b'\x00\x01\xad\x10\x00\x00\xdc\x8d\x1e""""\x92^\..."
9,10,10,FS-1418859,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.548333,-120.149167,5.0,USFS,CA,5,005,Amador,b'\x00\x01\xad\x10\x00\x00dS\\\xf2\x8b\t^\xc0\...


In [6]:
for index, row in fire.iterrows():
    row["LAT/LON"] = str(row['LATITUDE']) +','+ str(row['LONGITUDE'])

In [7]:
LAT_list = fire['LATITUDE'].to_list()
LON_list = fire['LONGITUDE'].to_list()
LAT_LON = []

In [8]:
for index,val in enumerate(LAT_list):
    LAT_LON.append(str(LAT_list[index])+','+str(LON_list[index]))


In [9]:
len(LAT_LON)

1880465

In [10]:
fire["LAT/LON"] = LAT_LON

In [11]:
fire["LAT/LON"].nunique()

1569708

In [12]:
LAT_LON_uni = fire["LAT/LON"].unique()
LAT_LON_uni 

array(['40.03694444,-121.00583333', '38.93305556,-120.40444444',
       '38.98416667,-120.73555556', ..., '40.481637,-122.389375',
       '37.67223469,-120.89835605', '34.263217,-116.83095'], dtype=object)

In [13]:
LAT_uni = []
LON_uni = []

for val in LAT_LON_uni:
    temp = val.split(',')
    LAT_uni.append(temp[0])
    LON_uni.append(temp[1])

In [14]:
len(LAT_uni)

1569708

In [15]:
import csv
with open('Data.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(['lat', 'long'])
    for val in LAT_LON_uni:
        temp = val.split(',')
        spamwriter.writerow([temp[0], temp[1]])

In [16]:
import csv
with open('Data.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    # for row in spamreader:
    #     print(row)

In [17]:
import numpy as np
import haversine as hs

fire.head()
df = pd.read_csv("Data.csv")

print(min(fire["LATITUDE"]))
print(max(fire["LATITUDE"]))
print(min(fire["LONGITUDE"]))
print(max(fire["LONGITUDE"]))

bottomLeft = (min(fire["LATITUDE"]), min(fire["LONGITUDE"]))
bottomRight = (min(fire["LATITUDE"]), max(fire["LONGITUDE"]))
topLeft = (max(fire["LATITUDE"]), min(fire["LONGITUDE"]))
topRight = (max(fire["LATITUDE"]), max(fire["LONGITUDE"]))

length = hs.haversine(bottomLeft,bottomRight)
width = hs.haversine(bottomLeft,topLeft)
print(length)
print(width)

cols = np.linspace(bottomLeft[1], bottomRight[1], num=6000)
rows = np.linspace(bottomLeft[0], topLeft[0], num=3000)

df['col'] = np.searchsorted(cols, df['long'])
df['row'] = np.searchsorted(rows, df['lat'])

df.to_csv('Data_with_category.csv')

17.93972222
70.3306
-178.8026
-65.25694444
11727.55765087666
5825.607858252318


In [34]:
data_frame = pd.read_csv("Data_with_category.csv")

data_frame["Grid"] = data_frame["col"].astype(str) + "," + data_frame["row"].astype(str)
data_frame.to_csv('Data_with_category.csv')

data_frame["Grid"].nunique()

654163

In [37]:
#Create one CSV out of large CSVs.
#Add 2021 data to the Data_Set directory and run below code
import csv, os
directory = "Data_Set"

index = 0
for filename in os.listdir(directory):
    file_path = 'Data_Set/' + filename
    with open(file_path, "r") as source:
        reader = csv.reader(source)
        
        with open("Data_Set_Edit/"+str(index)+".csv", "w") as result:
            writer = csv.writer(result)
            count = 0
            for r in reader:
                if (count>25):
                    writer.writerow(r)
                count += 1
    index += 1
